# Python Functions

In [ ]:
def add_one(number):
    return number + 1

add_one(2)

### First-Class Objects

In [ ]:
def say_hello(name):
    return f'Hello, {name}!'

def be_awesome(name):
    return f"Yo {name}, together we're the awesomest!"

def greet_bob(greeter_func):
    return greeter_func('Bob')

In [ ]:
greet_bob(say_hello)

In [ ]:
greet_bob(be_awesome)

### Inner Functions

In [ ]:
def parent():
    print('Printing from parent()')

    def first_child():
        print('Printing from first_child()')

    def second_child():
        print('Printing from second_child()')

    second_child()
    first_child()

What happens when you call the `parent()` function?

In [ ]:
# Prints
# - parent message
# - second child message
# - first child message
parent()

Can you call either `first_child()` or `second_child() **outside** `parent`?

In [ ]:
try:
    first_child()
except NameError as ne:
    print(f'Exception {type(ne).__name__}: {ne}')

### Functions as Return Values

In [ ]:
# Returns a **function** defined inside `parent()`
def parent(num):
    def first_child():
        return "Hi, I'm Elias"

    def second_child():
        return "Call me Ester"

    # Returns a **reference** to a function; that is,
    # the returned function is **not** executed.
    if num == 1:
        return first_child
    else:
        return second_child

In [ ]:
parent(1)

In [ ]:
parent(1)()

In [ ]:
parent(2)

In [ ]:
parent(2)()

If we remember the return values, we can **execute** the functions at a later time.

In [ ]:
first = parent(1)
second = parent(2)

In [ ]:
first()

In [ ]:
second()

## Simple Decorators in Python

An example

In [ ]:
def decorator(func):
    def wrapper():
        print('Something is happening **before** the function is called.')
        func()
        print('Something is happening **after** the function is called.')
    return wrapper

def say_whee():
    print('Whee!')

say_whee = decorator(say_whee)

In [ ]:
say_whee()



Remember, the so-called "decoration" happens when the line

`say_whee = decorator(say_whee)`

is executed. This line simply re-binds `say_whee` to the
inner function, `wrapper`, returned by `decorator()`.

In [ ]:
say_whee

Remember that `say_whee` now has a **refence** to the original `say_whee`
function as the value of argument, `func`. It calls the original `say_whee`
function between the two calls to `print()`.

Let's look at a second example.

In this example, the wrapper modifies the wrapped function **dynamically**.

Here's an example.

In [ ]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep.
    return wrapper

def say_whee():
    print('Whee!')

say_whee = not_during_the_night(say_whee)

The effect of `say_whee()` is **different** depending on the time of day.

For example, calling `say_whee()` between 7am and 10pm (0700 and 2200)
will print a "loud" message, 'Whee!'. But between 10pm (2200) and
7am (0700), calling `say_whee()` will do nothing (simply returning `None`).


In [ ]:
say_whee()

### Adding Syntactic Sugar

The following example accomplishes the exact same goal as our
"Simple Decorators in Python" section above.

In [ ]:
def decorator(func):
    def wrapper():
        print('Something is happening **before** the function is called.')
        func()
        print('Something is happening **after** the function is called.')
    return wrapper

@decorator
def say_whee():
    print('Whee!')

say_whee()

### Reusing Decorators

Let's define a new decorator.

As a technique to ease our memory load, we'll name the wrapper function
something like, `wrapper_<foo>` where `<foo>` is a placeholder containing
the name of the outermost function.

In [ ]:
# This decorator calls the decorated `func` twice.
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

In [ ]:
@do_twice
def say_whee():
    print('Whee!')
say_whee()

### Decorating functions with arguments

First, the **wrong** way.

In [ ]:
@do_twice
def greet(name):
    print(f'Hello, {name}!')

In [ ]:
try:
    greet('World')
except Exception as e:
    print(f'Exception {type(e).__name__}: {e}')

The problem is that the inner function, `wrapper_do_twice`, takes
**no** arguments, but we **actually** passed the argument "World."

One fix would be to add an argument to the `wrapper_do_twice()`
function; however, this change would cause an exception when
calling the decorated function, `say_whee()`.

The solution is to use `*args` and `**kwargs` in the inner wrapped
function. This construct allows the inner function to take **any**
number of positional and **any** number of keyword arguments.

Let's rewrite `wrapper_do_twice()` to expect these arguments.

In [ ]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

In [ ]:
@do_twice
def say_whee():
    print('Whee!')

say_whee()

In [ ]:
@do_twice
def greet(name):
    print(f'Hello, {name}!')

greet('World')

### Returning Values from Decorated Functions

What happens to the returned value of a decorated function?

That behavior is actually controlled by the **decorator**.

Let's see this in action.

In [ ]:
@do_twice
def return_greeting(name):
    print('Creating a greeting')
    return f'Hi, {name}!'

Now let's try to use `return_greeting`.

In [ ]:
hi_adam = return_greeting('Adam')

In [ ]:
print(hi_adam)

In this situation, our wrapper "ate" the returned value.

To repair this issue, our decorator **must** return any result from
the inner function. (For example, `wrapper_do_twice()`.)


Let('s change `wrapper_do_twice()`, the inner function of our decorator,
`do_twice()`

In [ ]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [ ]:
@do_twice
def greet(name):
    print('Creating a greeting')
    return f'Hi, {name}!'

This time, the result of the (second) wrapped function is returned!

In [ ]:
greet('Adam')

### Finding Yourself

How does one fit a decorated function into the Python ecosystem?
Specifically, how does one fit into Python introspection?

Here's an example.

In [ ]:
print

In [ ]:
print.__name__

In [ ]:
help(print)

This same behavior applies to functions that I write.

In [ ]:
def snafu():
    print('Situation normal, all fouled up.')

print(snafu)
print(snafu.__name__)
help(snafu)



However, notice that **after** decorating a function I wrote, the information
is "less" helpful.

In [ ]:
say_whee

In [ ]:
say_whee.__name__

In [ ]:
help(say_whee)

To fix this issue, we will use **another** decorator, `functools.wrap`,
to decorate our **wrapped** function.

In [ ]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [ ]:
@do_twice
def say_whee():
    print('Whee!')

In [ ]:
say_whee

In [ ]:
say_whee.__name__

In [ ]:
help(say_whee)

**Note:** The `@functools.wraps` decorator actually uses
`@functools.update_wrapper()` to update special attributes like
`__name__` and `__doc__` that are used in introspection.